In [3]:
import torch
import torch.nn as nn
import timm
import xgboost as xgb
import os
import pandas as pd
import numpy as np
import joblib
from sklearn import preprocessing
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import cv2
from tqdm import tqdm
import torch.nn.functional as F

print("lancement voting classifier 3 modeles")

# config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path_data = r"C:\Users\amisf\Desktop\datascientest_projet"
path_out = os.path.join(path_data, "implementation", "outputs")

# je recupere la liste des classes depuis le fichier train
# cest vital pour remettre les id dans le bon ordre a la fin
df_train = pd.read_csv(os.path.join(path_data, "data", "raw", "Y_train_CVw08PX.csv"))
if 'prdtypecode' not in df_train.columns: df_train = df_train.rename(columns={df_train.columns[1]: 'prdtypecode'})
le = preprocessing.LabelEncoder()
le.fit(df_train['prdtypecode'])
num_classes = len(le.classes_)
print(f"classes detectees : {num_classes}")

# 1. PREPARATION DONNEES TEST

# je charge le csv de test
path_test_csv = os.path.join(path_data, "data", "raw", "X_test_update.csv")
df_test = pd.read_csv(path_test_csv)

# je cherche les images
path_img = os.path.join(path_data, "data", "raw", "images", "images")
# fallback si dossier pas trouve
if not os.path.exists(path_img):
    for root, dirs, files in os.walk(path_data):
        if "images" in dirs: path_img = os.path.join(root, "images"); break

# je cree le chemin complet
df_test['path'] = df_test.apply(lambda r: os.path.join(path_img, f"image_{r['imageid']}_product_{r['productid']}.jpg"), axis=1)
print(f"images test a traiter : {len(df_test)}")

# dataset generique pour pytorch
class TestDataset(Dataset):
    def __init__(self, df, size, interpolation=cv2.INTER_LINEAR):
        self.paths = df['path'].values
        self.size = size
        self.inter = interpolation
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        try:
            img = cv2.imread(self.paths[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (self.size, self.size), interpolation=self.inter)
            # normalisation standard imagenet a la main
            img = img / 255.0
            img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
            # passage en chw
            img = img.transpose(2, 0, 1)
            return torch.tensor(img, dtype=torch.float32)
        except:
            return torch.zeros((3, self.size, self.size), dtype=torch.float32)


# 2. PREDICTION M1 : DINOv3 (LE VISIONNAIRE)

print("\n--- prediction m1 : dinov3 ---")

# config m1
size_m1 = 518
batch_m1 = 8 # attention vram

ds_m1 = TestDataset(df_test, size_m1, interpolation=cv2.INTER_CUBIC)
loader_m1 = DataLoader(ds_m1, batch_size=batch_m1, shuffle=False, num_workers=0)

# chargement architecture
model_m1 = timm.create_model('vit_large_patch14_reg4_dinov2.lvd142m', pretrained=False, num_classes=num_classes)
path_m1 = os.path.join(path_out, "M1_IMAGE_DeepLearning_DINOv3.pth")
model_m1.load_state_dict(torch.load(path_m1))
model_m1.to(device).eval()

# inference
probs_m1 = []
with torch.no_grad():
    for batch in tqdm(loader_m1, desc="m1 dino"):
        batch = batch.to(device)
        with torch.amp.autocast('cuda'):
            out = model_m1(batch)
            # je recupere les probas (softmax)
            probs = F.softmax(out, dim=1)
        probs_m1.append(probs.cpu().numpy())

probs_m1 = np.concatenate(probs_m1)
print(f"shape m1 : {probs_m1.shape}")

# nettoyage vram
del model_m1, loader_m1, ds_m1
torch.cuda.empty_cache()


# 3. PREDICTION M3 : EFFICIENTNET (LE RAPIDE)

# je fais m3 avant m2 car m2 a besoin de features
print("\n--- prediction m3 : efficientnet ---")

size_m3 = 224
batch_m3 = 64

ds_m3 = TestDataset(df_test, size_m3)
loader_m3 = DataLoader(ds_m3, batch_size=batch_m3, shuffle=False, num_workers=0)

model_m3 = models.efficientnet_b0(weights=None)
model_m3.classifier[1] = nn.Linear(1280, num_classes)
path_m3 = os.path.join(path_out, "M3_IMAGE_Classic_EfficientNetB0.pth")
model_m3.load_state_dict(torch.load(path_m3))
model_m3.to(device).eval()

probs_m3 = []
with torch.no_grad():
    for batch in tqdm(loader_m3, desc="m3 effnet"):
        batch = batch.to(device)
        out = model_m3(batch)
        probs = F.softmax(out, dim=1)
        probs_m3.append(probs.cpu().numpy())

probs_m3 = np.concatenate(probs_m3)
print(f"shape m3 : {probs_m3.shape}")

del model_m3
torch.cuda.empty_cache()

# 4. PREDICTION M2 : XGBOOST (LE CHAMPION)
print("\n--- prediction m2 : xgboost ---")
# attention xgboost a besoin de features
# je dois utiliser un resnet pour extraire les features dabord
# cest la methode qui nous a donne 85%

# extracteur resnet
extractor = timm.create_model('resnet50', pretrained=True, num_classes=0)
extractor.to(device).eval()

# loader pour features (taille 224 standard)
ds_feat = TestDataset(df_test, 224)
loader_feat = DataLoader(ds_feat, batch_size=64, shuffle=False, num_workers=0)

features_list = []
with torch.no_grad():
    for batch in tqdm(loader_feat, desc="extraction features m2"):
        batch = batch.to(device)
        feats = extractor(batch)
        features_list.append(feats.cpu().numpy())

x_test_features = np.concatenate(features_list)
print(f"features extraites : {x_test_features.shape}")

del extractor, loader_feat, ds_feat
torch.cuda.empty_cache()

# chargement xgboost
model_m2 = xgb.XGBClassifier()
model_m2.load_model(os.path.join(path_out, "M2_IMAGE_Classic_XGBoost.json"))

# prediction xgboost (predict_proba pour avoir les scores)
probs_m2_raw = model_m2.predict_proba(x_test_features)

# attention xgboost renvoie les probas dans l ordre de ses classes internes
# je dois verifier si je dois reordonner mais generalement cest bon si entraine avec le label encoder
# par securite je fais confiance a lordre du label encoder charge
probs_m2 = probs_m2_raw
print(f"shape m2 : {probs_m2.shape}")

# 5. VOTING (FUSION PONDEREE)
print("\n--- fusion des cerveaux ---")

# je definis les poids selon la puissance des modeles
# m2 (xgboost) est le boss (85%) -> poids 3
# m1 (dino) est fort (75%+) -> poids 2
# m3 (effnet) est le petit frere -> poids 1

w1 = 2.0 # dino
w2 = 3.0 # xgboost
w3 = 1.0 # effnet

# somme ponderee
final_probs = (w1 * probs_m1 + w2 * probs_m2 + w3 * probs_m3) / (w1 + w2 + w3)

# choix final (argmax)
final_preds_idx = np.argmax(final_probs, axis=1)

# je convertis les index en vrais codes produits (10, 2280, etc)
final_preds_code = le.inverse_transform(final_preds_idx)

# 6. EXPORT LIVRABLE
print("generation fichier submission...")

df_submission = pd.DataFrame({
    'imageid': df_test['imageid'],
    'productid': df_test['productid'],
    'prdtypecode': final_preds_code
})

path_sub = os.path.join(path_out, "submission_voting_m1_m2_m3.csv")
df_submission.to_csv(path_sub, index=False)

print(f"termine. fichier pret : {path_sub}")
print("on espere que ca va peter le score")

lancement voting classifier 3 modeles
classes detectees : 27
images test a traiter : 13812

--- prediction m1 : dinov3 ---


m1 dino: 100%|██████████| 1727/1727 [14:54<00:00,  1.93it/s]


shape m1 : (13812, 27)

--- prediction m3 : efficientnet ---


m3 effnet: 100%|██████████| 216/216 [00:19<00:00, 11.15it/s]


shape m3 : (13812, 27)

--- prediction m2 : xgboost ---


extraction features m2: 100%|██████████| 216/216 [00:38<00:00,  5.68it/s]


features extraites : (13812, 2048)
shape m2 : (13812, 27)

--- fusion des cerveaux ---
generation fichier submission...
termine. fichier pret : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs\submission_voting_m1_m2_m3.csv
on espere que ca va peter le score
